In [2]:
# import our usual things
%matplotlib inline
import cartopy
import pandas as pd
import matplotlib.pyplot as plt
import geopandas 
import ipyleaflet
import numpy as np
import bqplot
import ipywidgets
from bqplot import pyplot as plt

In [71]:
full = pd.read_csv('https://query.data.world/s/c5nj3jizdkifhlenho4iek26gycpgm')
colors = ["#F70020","#191A1A","#FB9701","#1A7D00","#072C8F","#9E9E9E"]

# data fetching and cleaning
def get_country_data(country):
    country_all = full[full['location'] == country]
    country_time = country_all[country_all['total_cases']>100]
    return country_time
# To plot a beautiful plot in bqplot we should have all the lines in same np.arry length
def transformToNp(arrays):
    max_len = max([len(a) for a in arrays])
    return np.array([np.concatenate((np.array(a),np.full((max_len-len(a), ), np.nan))) for a in arrays])
def generate_bar(countries,confirm,dayNo='last day'):
    ys = []

    for ind,country in enumerate(countries):
        choose = 'total_cases' if confirm else 'total_deaths'
        ys.append(get_country_data(country)[choose])
    # ys = transformToNp(ys)
    # print([ for i in ys])
    x_sc_bar = bqplot.OrdinalScale()
    y_scLinear = bqplot.LinearScale()

    bqbar = bqplot.Bars(
        x=countries,y=[np.array(i)[-1] for i in ys],scales = {'x': x_sc_bar, 'y': y_scLinear},colors=colors,
                orientation='horizontal',)
    # add bar labels
    bqbar.label_display = True
    bqbar.label_display_format = '.0f'
    bqbar.label_font_style = {'fill': 'white', 'font-size': '20px'}
    ax_x = bqplot.Axis(scale=x_sc_bar, orientation="vertical")
    ax_y = bqplot.Axis(scale=y_scLinear)
    margin = dict(top=5, bottom=0, left=100, right=5)
    fig = bqplot.Figure(marks = [bqbar], axes = [ax_x, ax_y],legend_location='top-left',fig_margin=margin,title=dayNo)
    fig.layout.height = '300px'
    fig.layout.width = '900px'
    
    return fig


In [81]:
colors = ["#F70020","#191A1A","#FB9701","#1A7D00","#072C8F","#9E9E9E"]

class legendWidget(object):
    """A legend Widget using a horizontal bar chart
    
    marks: line marks from a bqplot figure.  
    
    These line marks must have legend labels 
    (in line mark, remove other legend by using this: display_legend = False)
    e.g. >>> legend = legendWidget(fig.marks) 
    
    BQplot module imported as bq (import bqplot as bq)

    """
    def __init__(self, marks, data=list()):
        """Return a new Legend object."""
        y_ord = bqplot.OrdinalScale()
        x_sc = bqplot.LinearScale()
        
        legendLabels = []
        colours = []
        markLineNums = [] # record number of lines per mark

        for mark in marks:        
            legendLabels += mark.labels
            colours += mark.colors[:len(mark.labels)]
            markLineNums.append(len(mark.labels))  
        
        
        bar = bqplot.Bars(
            y=[1]*len(legendLabels) , # all bars have a amplitude of 1
            x=legendLabels, 
            scales={'y': x_sc, 'x': y_ord},
            colors=colours ,
            padding = 0.6,
            orientation='horizontal',
            stroke = 'white'  #remove the black border around the bar
            )
        
        ax_y = bqplot.Axis(scale=y_ord, orientation="vertical")
        ax_x = bqplot.Axis(scale=x_sc)
        ax_x.visible = False
        margin = dict(top=40, bottom=0, left=110, right=5)
        barFig = bqplot.Figure(marks=[bar], axes=[ax_y, ax_x], fig_margin=margin)
        
        # Variable height depending on number of bars in legend
        barFig.layout.height = str(45 + 20 * len(legendLabels)) + 'px'
        barFig.layout.width = '170px'

        barFig.min_aspect_ratio = 0.000000000001 # effectively remove aspect ratio constraint
        barFig.max_aspect_ratio = 999999999999999 # effectively remove aspect ratio constraint
        barFig.background_style = {'fill': 'White'}   
                    
        self.fig = barFig
        self.bar = bar
        self.colours = colours
        self.markLineNums = markLineNums


In [182]:
# Add another bar graph
colors = ["#F70020","#191A1A","#FB9701","#1A7D00","#072C8F","#9E9E9E"]

def make_line_chart(countries, log, confirm):
    x_sc = bqplot.LinearScale(min=1)
    # lets do one y-scale over linear and 1 over log
    y_scLinear = bqplot.LinearScale(min=100)
    y_scLog = bqplot.LogScale()
    y_sc = y_scLog if log else y_scLinear
    
    indexes = range(1,51)
    lines = []
    def_tt = bqplot.Tooltip(fields=['x', 'y','name'], formats=['', '.0f',''], labels=['Country', 'Number','c'])
    tooltip_out = ipywidgets.Output()
    legend_out = ipywidgets.Output()
    
    ys = []
    for ind,country in enumerate(countries):
        choose = 'total_cases' if confirm else 'total_deaths'
        ys.append(np.fromiter(get_country_data(country)[choose], dtype="int"))
    ys = transformToNp(ys)
 
    class HoverTrigger:
        def __init__(self):
            self.name = ''
            self.number = 0
            self.days = 0
    hover_trigger = HoverTrigger()
    
    def update_bar(day,bar_y,dayNo):

        fig_bar.marks[0].y = bar_y
        fig_bar.title = 'Day'+ str(dayNo)
        
    def hover_handler(m, hover_event):
        tooltip_out.clear_output()

        hover_data = hover_event['data']
        hover_x = int(hover_data['x'])-1
        rows = []
        bar_y = []
        for ind,country in enumerate(countries):
            if np.isnan(ys[ind][hover_x]):
                rows.append([country,'Not Existed'])
                bar_y.append(0)
            else:
                rows.append([country,int(ys[ind][hover_x])])
                bar_y.append(ys[ind][hover_x])
        update_bar(hover_data['x'],bar_y,hover_data['x'])
#         with tooltip_out:
# #             print(html_table.value)
#             display(html_table)
        

    for ind,country in enumerate(countries):
        
        bqScatter = bqplot.Scatter(x = indexes, y = ys[ind], 
                    scales = {'x': x_sc, 'y': y_sc},marker='circle',marker_size=16,colors=[colors[ind]])
        bqScatter.on_hover(hover_handler)
        lines.append(bqScatter)
    
    bqlines = bqplot.Lines(x = indexes, y = ys, 
                scales = {'x': x_sc, 'y': y_sc},colors=colors,display_legend=False, labels=countries,stroke_width=3)
    
    

    lines.append(bqlines)
    
    
    # and lets plot an x axis like before
    ax_x = bqplot.Axis(scale = x_sc, label = 'Day',tick_values=np.array([i for i in range(1,51,1)]))
    # y axis for linear/log presentation
    ax_y = bqplot.Axis(scale = y_sc, label = 'Cases', 
                        orientation = 'vertical')
    # lets allow pan and zoom

    selector = bqplot.interacts.IndexSelector(line_width=1,scale = x_sc) 
    def print_change(change):
        l.value = str(change)
    margin = dict(top=0, bottom=20, left=100, right=5)
    fig = bqplot.Figure(marks = lines, 
                        axes = [ax_x, ax_y],legend_location='top-left',fig_margin=margin)

#     print(len(fig.marks),'length')
#     for i in fig.marks:
#         print(i)
    legend_bar = legendWidget(fig.marks,ys)

    def changeOpacity(self, target):
        """Enable legend interactivity. 
        Use in conjunction with class legendWidget(object) 
        Click on legend bar to toggle opacity of all other lines

        """

        # I'm not sure how to pass in the line chart and legend widgets from on_element_click(). 
        # Need to explicity define them below.
        lineFig = fig  # set lineFig to name of line chart figure
        legendFig = legend_bar  # set legendFig to name of new legend widget

        opacity = 0.1   # set opacity of non selected lines here
        sigNum = target['data']['index']
        bar = self
        if bar.opacities == [] or bar.opacities[sigNum] == opacity:
            bar.opacities=[opacity]*sigNum + [1] + [opacity]*(len(bar.x) - sigNum - 1)        

            # Some marks in line plot have more than 1 line.  
            currentLineNum=0
            for markNum,markLineNum in enumerate(legendFig.markLineNums):
                lineFig.marks[markNum].opacities = bar.opacities[currentLineNum:currentLineNum + markLineNum]
#                 print(bar.opacities[currentLineNum:currentLineNum + markLineNum])
                currentLineNum+=markLineNum
            # change scatter opacity
            for markNum in range(currentLineNum):
#                 print(bar.opacities[markNum])
                # defulat_opacity ???? why ????? 
                lineFig.marks[markNum].default_opacities = np.array([bar.opacities[markNum]]) 
#                 lineFig.marks[markNum].default_opacities = [0]
        else:
            bar.opacities = []
            for mark in lineFig.marks:
    #         for mark in self:
                mark.opacities = []
                mark.default_opacities = []
        
        
    legend_bar.fig.marks[0].on_element_click(changeOpacity)
    
#     fig_bar = bqplot.Figure(marks = [bqbar], 
#                         axes = [ax_x, ax_y],legend_location='top-left')
    
    #display(fig)
    l = ipywidgets.Label()
#     db_index = ipywidgets.HTML(value='')
    temp = ipywidgets.HTML(value='')
    legend_temp = ipywidgets.HTML(value='')
    fig_bar = generate_bar(countries,confirm)
#     ipywidgets.jslink((fig.interaction, 'selected'),(temp,'value'))
#     ipywidgets.interact(my_function, a=temp)
    return ipywidgets.VBox([legend_out,legend_bar.fig,fig,fig_bar])

In [183]:
from IPython.display import display, clear_output
# f = make_line_chart(['China','United States','Italy'],True)
%config InlineBackend.close_figures=False

# plt.ioff()
# plt.close('all') # if you get a "too many figures open" warning
default_countries = ['China','United States','Italy']
colors = ["#F70020","#191A1A","#FB9701","#1A7D00","#072C8F","#9E9E9E"]

country_select = ipywidgets.SelectMultiple(
    options=['China','United States','Italy','United Kingdom','France','India','South Korea', 'Japan'],
    value=default_countries,
    #rows=10,
    description='Countries',
    disabled=False
)
confirm_select = ipywidgets.ToggleButtons(
    options=['Confirmed', 'Death'],
#     description='Speed:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['Confirmed Cases', 'Death Cases'],
#     icons=['check'] * 3
)

log_select = ipywidgets.Checkbox(
    value=True,
    description='Log',
    disabled=False,
    indent=True
)

check = ipywidgets.Button(
    description='Click me to update',
    disabled=False,
    button_style='', 
    tooltip='Change'
)
selects = ipywidgets.HBox([confirm_select,log_select,check])
controls = ipywidgets.VBox([country_select,selects])
# controls.layout.top = '200px'

out = ipywidgets.Output()  # NEW WIDGET CALL

# print('1')


def update(change):
    if confirm_select.value == 'Confirmed':
        c_s = True
    else:
        c_s = False
    countries = country_select.value
    fig = make_line_chart(countries, log_select.value, c_s)
    with out:
        clear_output()
        display(fig)
#         display(label)
vbox = ipywidgets.VBox([controls,out])
display(vbox)
check.on_click(update)
update(None)
# display(vbox)

In [381]:
countries = ['China','United States','Italy','France','Germany']
# def moving_average(a, n=3) :
#     ret = np.cumsum(a,dtype="int")
#     ret[n:] = ret[n:] - ret[:-n]
#     return ret[n - 1:] / n

def generate_line_confirm_new(countries,index):

    lines = []
    xs = []
    ys = []
    for ind,country in enumerate(countries):
        xs.append(np.fromiter((get_country_data(country).iloc[:-51+index,4]), dtype="int"))
        ys.append(np.fromiter(get_country_data(country).iloc[:-51+index,2], dtype="int"))
#     rolling_xs = []
#     for x in enumerate(xs):
#         new_xs = np.zeros(len(ys[0]))
#         for ind,i in enumerate(x):
#             if ind < len(x) - 2:
#                 new_xs[ind] = (xs[ind]+xs[ind+1]+xs[ind+2])//3
#         rolling_xs.append(new_xs)
        
    ys = transformToNp(ys)
    xs = transformToNp(xs)
    x_sc = bqplot.LogScale(min=100,max=8*10**5)
    # lets do one y-scale over linear and 1 over log
    y_sc = bqplot.LogScale(min=10,max=50000)

    ax_x = bqplot.Axis(scale = x_sc, label = 'Existed Cases')
    # y axis for linear/log presentation
    ax_y = bqplot.Axis(scale = y_sc, label = 'New Cases', 
                        orientation = 'vertical')
    # for ind,country in enumerate(countries):
    #     x_d = get_country_data(country).iloc[:,4]
    #     y_d = get_country_data(country).iloc[:,2]

    line = bqplot.Lines(x = xs, y = ys, 
                        scales = {'x': x_sc, 'y': y_sc},colors=colors,display_legend=False,stroke_width=2.5,labels=countries)
    lines.append(line)
    fig = bqplot.Figure(marks = lines, 
                            axes = [ax_x, ax_y],legend_location='top-left')
    legend_bar = legendWidget(fig.marks)

    def changeOpacity(self, target):
        """Enable legend interactivity. 
        Use in conjunction with class legendWidget(object) 
        Click on legend bar to toggle opacity of all other lines

        """

        # I'm not sure how to pass in the line chart and legend widgets from on_element_click(). 
        # Need to explicity define them below.
        lineFig = fig  # set lineFig to name of line chart figure
        legendFig = legend_bar  # set legendFig to name of new legend widget

        opacity = 0.1   # set opacity of non selected lines here
        sigNum = target['data']['index']
        bar = self
        if bar.opacities == [] or bar.opacities[sigNum] == opacity:
            bar.opacities=[opacity]*sigNum + [1] + [opacity]*(len(bar.x) - sigNum - 1)        

            # Some marks in line plot have more than 1 line.  
            currentLineNum=0
            for markNum,markLineNum in enumerate(legendFig.markLineNums):
                lineFig.marks[markNum].opacities = bar.opacities[currentLineNum:currentLineNum + markLineNum]
#                 print(bar.opacities[currentLineNum:currentLineNum + markLineNum])
                currentLineNum+=markLineNum

        else:
            bar.opacities = []
            for mark in lineFig.marks:
    #         for mark in self:
                mark.opacities = []
                mark.default_opacities = []


    legend_bar.fig.marks[0].on_element_click(changeOpacity)

    fig.fig_margin = dict(top=20, bottom=50, left=50, right=20)
    ret = ipywidgets.VBox([legend_bar.fig,fig])
    return ret,fig,fig.marks[0].x,fig.marks[0].y


In [401]:
data_setx = [0]
data_sety = [0]

def generate_dataset():
    for i in range(1,51):
        f,fig,xs,ys = generate_line_confirm_new(countries,i)
        data_setx.append(xs)
        data_sety.append(ys)
generate_dataset()

In [404]:

f,fig,xs,ys = generate_line_confirm_new(countries,30)

int_slider = ipywidgets.IntSlider(
    value=50,
    min=1,
    max=50,
    step=1,
    description='Test:',
    disabled=False,
    continuous_update=True,
    readout=True,
    readout_format='d',
)
def slide_hanler(event):
    new_val = event['new']
    fig.marks[0].x = data_setx[new_val]
    fig.marks[0].y = data_sety[new_val]
int_slider.observe(slide_hanler,'value')
play = ipywidgets.Play(interval = 50, value = 50, min = 1, max = 50, step = 1, description = "Press Play")
ipywidgets.jslink((play, 'min'), (int_slider, 'min'))
ipywidgets.jslink((play, 'max'), (int_slider, 'max'))
ipywidgets.jslink((play, 'value'), (int_slider, 'value'))
int_slider.layout.width = '100%'
display(ipywidgets.VBox([f,ipywidgets.HBox([play, int_slider])]))